In [ ]:
import cv2
from ultralytics import YOLO
import time

# Cargar el modelo YOLOv8 entrenado
model = YOLO('best.pt')

# Conectar la cámara (usando video0)
cap = cv2.VideoCapture(0)

# Verificar si la cámara se abrió correctamente
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

# Establecer el tamaño del frame
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)

# Crear la ventana para mostrar el video
cv2.namedWindow("Detección de animales salvajes", cv2.WINDOW_NORMAL)

while True:
    # Leer un frame de la cámara
    ret, frame = cap.read()

    # Verificar si se capturó el frame correctamente
    if not ret:
        print("Error: No se pudo capturar el frame.")
        break

    # Registrar el tiempo de inicio
    start_time = time.time()

    # Aplicar el modelo YOLO para hacer las detecciones
    results = model(frame)

    # Procesar y dibujar las cajas de detección en el frame
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            class_id = int(box.cls[0])
            label = f'{model.names[class_id]} {confidence:.2f}'

            # Dibujar la caja alrededor del objeto detectado
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Mostrar el frame con las detecciones
    cv2.imshow("Detección de animales salvajes", frame)

    # Calcular el tiempo de procesamiento
    processing_time = time.time() - start_time
    fps = 1 / processing_time  # Calcula los fps actuales

    # Imprimir los fps cada 5 segundos
    if int(time.time()) % 5 == 0:
        print(f'FPS: {fps:.2f}')

    # Salir al presionar la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera la cámara y cierra la ventana
cap.release()
cv2.destroyAllWindows()
